In [1]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import requests
import time
import numpy as np
import io
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.models as models
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import random
from tqdm import tqdm
import json
from torch.optim.lr_scheduler import CosineAnnealingLR
import threading
import torchvision.models as models
import torch.nn as nn
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel
from nltk.corpus import wordnet
from caption_transforms import SimCLRData_Caption_Transform
from image_transforms import SimCLRData_image_Transform
from dataset import FlickrDataset,Flickr30kDataset
from models import ResNetSimCLR,OpenAI_SIMCLR
from utils import get_gpu_stats,layerwise_trainable_parameters,count_trainable_parameters,get_gpu_memory
from metrics import inter_ContrastiveLoss, intra_ContrastiveLoss
from metrics import LARS,Optimizer_simclr
from logger import Logger
from train_fns import train, test


In [2]:
!nvidia-smi

Thu Apr 13 01:47:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:3B:00.0 Off |                  Off |
| N/A   35C    P0    36W / 250W |  12153MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                  Off |
| N/A   

In [2]:
torch.cuda.empty_cache()
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.current_device())


0


In [ ]:
torch.cuda

In [4]:
dataset = Flickr30kDataset('/work/08629/pradhakr/maverick2/cv_project/flickr30k-images', 
                           '/work/08629/pradhakr/maverick2/cv_project/flickr30k_captions/results_20130124.token',
                           caption_index_1=0,
                           caption_index_2=1,
                          image_transform=SimCLRData_image_Transform())
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [29783, 1000, 1000])
batch_size = 16
train_loader = DataLoader(train_set, 
                         batch_size=batch_size, 
                         shuffle=True, 
                         num_workers=4, 
                         pin_memory=True)
val_loader = DataLoader(val_set, 
                         batch_size=batch_size, 
                         shuffle=False, 
                         num_workers=4, 
                         pin_memory=True)
print(len(train_loader))
print(len(val_loader))

1862
63


In [ ]:
for idx ,batch in enumerate(val_loader):
    s=batch
    break

In [ ]:
resnet_model = ResNetSimCLR(
    model='resnet50',
    intra_projection_dim=128,
    inter_projection_dim =1024,
    layers_to_train=['layer3','layer4'],
    evaluate=False
).to(device)
gpt_model = OpenAI_SIMCLR(
    model='openai-gpt',
    intra_projection_dim=128,
    inter_projection_dim=1024,
    layers_to_train=['h.10','h.11'],
    evaluate=False
).to(device)

In [ ]:
intra_image,inter_image=resnet_model(s[1],device)
intra_image1,inter_image1=resnet_model(s[2],device)
intra_cap,inter_cap=gpt_model(s[3],device)
intra_cap1,inter_cap1=gpt_model(s[4],device)

In [ ]:
intra_loss=intra_ContrastiveLoss(device,temperature=0.07)
newinter_loss=inter_ContrastiveLoss(margin=0.2, max_violation=True)
print(intra_loss(intra_image,intra_image1, batch_size))
a,b=newinter_loss(inter_image,inter_image1,inter_cap,inter_cap1)
print(a,b)

In [5]:
def train1(dataloader,data_type, image_model, text_model, optimizer_image, optimizer_text, intra_criterion,inter_criterion,device,
          scheduler_image=None, scheduler_text=None, trade_off_ii=1, trade_off_cc=1,trade_off_ic=1,trade_off_ci=1):
    loss_epoch = 0

    for idx, batch in tqdm(enumerate(dataloader)):
        image_model.train()
        text_model.train()
        intra_contrastive_loss=0
        batch_size = batch[0].shape[0]
        if data_type=='flickr_travel':
            image1, image2, caption1, caption2 = batch[0], batch[1], batch[3], batch[4]
        if data_type=='flickr30k':
            image1, image2, caption1, caption2 = batch[1], batch[2], batch[3], batch[4]
            
        intra_image,inter_image = image_model(image1, device)
        intra_image1,inter_image1 = image_model(image2, device)
        intra_contrastive_loss+=(trade_off_ii * intra_criterion(intra_image, intra_image1, batch_size))
        del intra_image , intra_image1
        intra_cap,inter_cap = text_model(caption1, device)
        intra_cap1,inter_cap1 = text_model(caption2, device)
        intra_contrastive_loss+=(trade_off_cc * intra_criterion(intra_cap, intra_cap1, batch_size))
        #intra_contrastive_loss = (trade_off_ii * intra_criterion(intra_image, intra_image1, batch_size) +
                            #trade_off_cc * intra_criterion(intra_cap, intra_cap1, batch_size))
            
            
        ci_loss, ic_loss=inter_criterion(inter_image,inter_image1,inter_cap,inter_cap1)
        del  inter_image,inter_image1,inter_cap,inter_cap1
        inter_contrastive_loss= trade_off_ci*ci_loss + trade_off_ic*ic_loss
        
        total_loss = intra_contrastive_loss + inter_contrastive_loss
        
        total_loss.backward()
        optimizer_image.step()
        optimizer_text.step()

        optimizer_image.zero_grad()
        optimizer_text.zero_grad()
        print(total_loss.item())
        loss_epoch += total_loss.item()
        #del batch, intra_image, inter_image, intra_image1, inter_image1, intra_cap
        #del inter_cap, intra_cap1, inter_cap1, intra_contrastive_loss, ci_loss, ic_loss, inter_contrastive_loss
    if scheduler_image:
        scheduler_image.step()
    if scheduler_text:
        scheduler_text.step()
    epoch_loss = loss_epoch / len(dataloader)
    return epoch_loss
def test1(dataloader, data_type, image_model, text_model,intra_criterion,inter_criterion, device, trade_off_ii=1,
         trade_off_cc=1,trade_off_ic=1,trade_off_ci=1):

    loss_epoch = 0

    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            image_model.eval()
            text_model.eval()
            batch_size = batch[0].shape[0]
            if data_type=='flickr_travel':
                image1, image2, caption1, caption2 = batch[0], batch[1], batch[3], batch[4]
            if data_type=='flickr30k':
                image1, image2, caption1, caption2 = batch[1], batch[2], batch[3], batch[4]
            

            intra_image,inter_image = image_model(image1, device)
            intra_image1,inter_image1 = image_model(image2, device)
            intra_cap,inter_cap = text_model(caption1, device)
            intra_cap1,inter_cap1 = text_model(caption2, device)

            intra_contrastive_loss = (trade_off_ii * intra_criterion(intra_image, intra_image1, batch_size) +
                                trade_off_cc * intra_criterion(intra_cap, intra_cap1, batch_size))


            ci_loss, ic_loss=inter_criterion(inter_image,inter_image1,inter_cap,inter_cap1)
            inter_contrastive_loss= trade_off_ci*ci_loss + trade_off_ic*ic_loss

            total_loss = intra_contrastive_loss + inter_contrastive_loss
            print(total_loss.item())
            loss_epoch += total_loss.item()

            #del batch, intra_image, inter_image, intra_image1, inter_image1, intra_cap
            #del inter_cap, intra_cap1, inter_cap1, intra_contrastive_loss, ci_loss, ic_loss, inter_contrastive_loss            
            torch.cuda.empty_cache()

    epoch_loss = loss_epoch / len(dataloader)
    return epoch_loss

In [6]:
projection_dim=128
encoder_last_layer=2048
image_learning_rate = 0.001
text_learning_rate=4e-5
momentum = 0.9
temperature = 0.07
weight_decay = 0.0001
optimizer_type = 'sgd'
total_epochs=100
trade_off_ii=1
trade_off_cc=1
trade_off_ic=1e-4
trade_off_ci=1e-4
image_layers=['layer3','layer4']
resnet_model = ResNetSimCLR(
    model='resnet50',
    intra_projection_dim=128,
    inter_projection_dim =1024,
    layers_to_train=image_layers,
    evaluate=False
).to(device)
print('resnet_trainable_Params',count_trainable_parameters(resnet_model))
get_gpu_memory()
gpt_model = OpenAI_SIMCLR(
    model='openai-gpt',
    intra_projection_dim=128,
    inter_projection_dim=1024,
    layers_to_train=['h.10','h.11'],
    evaluate=False
).to(device)
print('gpt_trainable_Params',count_trainable_parameters(gpt_model))
get_gpu_memory()
# Define loss function
intra_loss=intra_ContrastiveLoss(device,temperature=0.07)
newinter_loss=inter_ContrastiveLoss(margin=0.2, max_violation=True)
# Define optimizers and schedulers
optimizer_image = Optimizer_simclr(optimizer_name=optimizer_type,
                                   model_parameters=resnet_model.parameters(),
                                   lr=image_learning_rate,
                                   momentum=momentum,
                                   weight_decay=weight_decay)

scheduler_image = optimizer_image.scheduler
optimizer_image = optimizer_image.optimizer

optimizer_text = Optimizer_simclr(optimizer_name=optimizer_type,
                                  model_parameters=gpt_model.parameters(),
                                  lr=text_learning_rate,
                                  momentum=momentum,
                                  weight_decay=weight_decay)

scheduler_text = optimizer_text.scheduler
optimizer_text = optimizer_text.optimizer


resnet_trainable_Params 32816256
Free GPU memory: 16.64 GB


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


gpt_trainable_Params 16242816
Free GPU memory: 15.64 GB


In [8]:
for epoch in range(100):
    start = time.time()
    train_loss = train1(dataloader=train_loader, 
                           data_type='flickr30k',
                           image_model=resnet_model, 
                           text_model=gpt_model,
                           optimizer_image=optimizer_image, 
                           optimizer_text=optimizer_text, 
                           intra_criterion=intra_loss,
                           inter_criterion=newinter_loss,
                            device=device,
                           scheduler_image=scheduler_image,
                           scheduler_text=scheduler_text,
                           trade_off_ii=trade_off_ii, 
                           trade_off_cc=trade_off_cc,
                           trade_off_ic=trade_off_ic,
                           trade_off_ci=trade_off_ci)
    test_loss = test(dataloader=val_loader, 
                     data_type='flickr30k',
                     image_model=resnet_model,
                     text_model=gpt_model,
                     intra_criterion=intra_loss,
                     inter_criterion=newinter_loss,
                     device=device,
                     trade_off_ii=trade_off_ii,
                     trade_off_cc=trade_off_cc,
                     trade_off_ic=trade_off_ic,
                     trade_off_ci=trade_off_ci)
    end = time.time()
    print('trainloss',round(train_loss,3),'testloss',round(test_loss,3),'time',round(end-start,1))

2it [00:02,  1.16it/s]

4.795647621154785
4.735579967498779


4it [00:02,  2.55it/s]

5.295603275299072
4.997143745422363


6it [00:03,  2.31it/s]

...

1.779638409614563
1.9618968963623047


144it [00:55,  3.18it/s]

2.109067916870117


144it [00:56,  2.55it/s]


KeyboardInterrupt: 

In [ ]:
!nvidia-smi

In [ ]:
index=7
print(a[index])
print(b[index])

In [ ]:
print(b[index])